Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [280]:
CITIES = pd.read_csv('../CI2024_LAB2/cities/us.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Abilene,32.454514,-99.738147
1,Akron,41.080456,-81.521429
2,Albuquerque,35.105552,-106.647388
3,Alexandria,38.818343,-77.082026
4,Allen,33.107224,-96.674676


## Lab3

In [281]:
median = np.median(DIST_MATRIX.reshape(1, -1))

DIST_MATRIX[DIST_MATRIX > median] = np.inf
G = nx.Graph()
for c1, c2 in combinations(CITIES.itertuples(), 2):
    G.add_node(c1.Index)
    G.add_node(c2.Index)
    if DIST_MATRIX[c1.Index, c2.Index] <= median:
        G.add_edge(c1.Index, c2.Index, weight=DIST_MATRIX[c1.Index, c2.Index])
nx.is_connected(G)

False

## Greedy Approach

In [285]:
def greedy_backtrack(graph: nx.Graph, current, end, visited, path, current_cost, finished):
        # Add current node to path and mark as visited
        path.append(current)
        visited.add(current)

        # If we reached the end node, this is the chosen path
        if current == end:
            return True, path, current_cost
        else:
            # Explore neighbors using a greedy strategy, smaller geodesic distance from end
            neighbors = G[current]
            neighbors = sorted(neighbors, key=lambda x: geodesic((CITIES.loc[x].lat, CITIES.loc[x].lon), (CITIES.loc[end].lat, CITIES.loc[end].lon)).km)
            neighbors_with_weights = [(neighbor, G[current][neighbor]['weight']) for neighbor in neighbors if neighbor not in visited]
            for neighbor, weight in neighbors_with_weights:
                
                current_cost += weight
                finished = greedy_backtrack(graph, neighbor, end, visited, path, current_cost + weight, finished)
                if finished:
                    return True, path, current_cost
        # Backtrack: unmark the node and remove it from the path
        visited.remove(current)
        path.pop()
        return False

In [286]:
def shortest_path(graph, start, end):
        best_path = []
        visited = set()
        _, best_path, best_cost = greedy_backtrack(graph, start, end, visited, [], 0, False)
        return best_path, best_cost

In [302]:
#Try Greedy Backtrack
#randomly select a node
start_node = np.random.choice(list(G.nodes))

#destination node
destination_node = np.random.choice(list(G.nodes))
while destination_node == start_node:
    destination_node = np.random.choice(list(G.nodes))

logging.info(f"Start node: {start_node}, Destination node: {destination_node}")
path, cost = shortest_path(G, int(start_node), destination_node)
logging.info(f"Path: {path}, Cost: {cost}")

INFO:root:Start node: 133, Destination node: 172
INFO:root:Path: [133, 172], Cost: 1376.3380693015297
